In [21]:
import os,re
import numpy as np
import subprocess
import datetime

In [ ]:
def make_geom_file(homedir, t=0.01918,U = 0.36):
    # this function modifies the already existing geom file

    geom_file =  open(homedir+"/cubic.geom", "r")
    geom_lines = geom_file.readlines()
    geom_file.close()

    geom_lines[13] = '0 0 1.0 0.0 0.0 '+str(t)+' '+ str(t)+' 0.0\n'
    geom_lines[14] = '0 0 0.0 1.0 0.0 '+str(t)+' '+ str(t)+' 0.0\n'
    geom_lines[15] = '0 0 0.0 0.0 1.0 '+str(t)+' '+ str(t)+' 0.0\n'
    geom_lines[16] = '0 0 0.0 0.0 0.0 0.0 0.0 '+str(U)+'\n'


    geom_file = open("cubic.geom", "w")
    geom_file.writelines(geom_lines)
    geom_file.close()
    
    return





def make_in_file(homedir, mu_up = 0,mu_down = 0,L =50, dtau = 0.01):
    #function modifies an already existing in file
    
    in_file =  open(homedir+"/in", "r")
    in_lines = in_file.readlines()
    in_file.close()


    in_lines[11] = 'mu_up  =  '+str(mu_up)+'\n'
    in_lines[12] = 'mu_dn  =  '+str(mu_down)+'\n'
    in_lines[13] = 'L      =  '+str(L)+'\n'
    in_lines[14] = 'dtau   =  '+str(dtau)+'\n'

    in_file =  open(homedir+"/in", "w")
    in_file.writelines(in_lines)
    in_file.close()

    in_file =  open(homedir+"/in", "r")
    in_lines = in_file.readlines()
    
    return

def extract_num(s):
    match_number = re.compile('-?[0-9]+\.?[0-9]*(?:[Ee]\ *?[-\+]\ *[0-9]+)?')
    nums = [float(x) for x in re.findall(match_number, s)]
    #returns the first two numbers from a string (value and error)
    return (nums[0],nums[1])
    


def read_out_file(homedir,file = 'test.out'):
    
    #reads tes.out file and extracts the results
    out_file = open(homedir+"/"+outfile, "r")
    out_lines = out_file.readlines()
    out_file.close()
    
    results = []
    #Density
    density = extract_num(out_lines[33])
    results.extend(density)
    
    #Doublon Density
    doub_density = extract_num(out_lines[44])
    results.extend(doub_density)

    #Average U
    Avg_U = extract_num(out_lines[30])
    results.extend(Avg_U)
    
    #Kinetic Energy
    Avg_KE= extract_num(out_lines[31])
    results.extend(Avg_KE)
    
    #Total Energy
    Avg_KE= extract_num(out_lines[32])
    results.extend(Avg_KE)

    return results

    

In [ ]:

homedir = "/home/k-rb/Documents/QMC/quest-qmc-mkl/EXAMPLE/geom"

folders = ["10Er-10G"]
ts = [0.0191824]
Us = [0.3636730]
os.path.exists(homedir+'/in')
    

Tstart = 0.212 #start temperature in Er
Tend = 1.01 # end temperatur in Er
deltaT = 0.041 #temperatur step size

mustart = -3.021 #start chemical potential
muend = 3.021 #end chemical potential
deltamu = 0.0402 #chemical potential stepsize

for i in range(len(folders)):

    # Make the directory for saving the results
    directory = "cubic-results-"+folders[i]
    path = os.path.join(homedir, directory)
    os.makedirs(path,exist_ok=True)
    
    
    #Create a fresh output file with resutls
    outputfile = homedir+'/'+directory+"/calcoutput_vv.txt"
    try:
        os.remove(outputfile)  
    except FileNotFoundError:
        pass

    columns = ["tempbeta","mu","density","err","doublon","err"]
    with open(outputfile, "ab") as f:
        np.savetxt(f, columns, fmt='%s', newline="\t",delimiter ='\t')


    """geom file"""
    make_geom_file(homedir,t = ts[i],U = Us[i])

    Templist = np.arange(Tstart,Tend,deltaT)
    mulist = np.arange(mustart,muend,deltamu)

    for Temp in Templist:

        betaTemp = 1/Temp
        LTemp = 50
        dbetaTemp = betaTemp/50

        for mu in mulist:
            print('T = ',Temp,'mu= ',mu,'started at ',datetime.datetime.now())

            """in file"""
            make_in_file(homedir, mu_up = mu,mu_down = mu,
                          L =LTemp, dtau = dbetaTemp)

            """ Perform the QMC simulation"""
            os.system("cd "+homedir+"/; ./ggeom in")

            """Grab the QMC results"""
            outfile = 'test.out'
            results = read_out_file(homedir,file = outfile)
            
            os.remove(homedir+"/"+outfile)


            results = [betaTemp,mu]+results
            results = np.array(results,dtype ='float')

            """Write Results to file"""
            with open(outputfile, "ab") as f:
                f.write(b"\n")
                np.savetxt(f, results[0:6], fmt='%1.4e', 
                           newline="\t",delimiter ='\t')

T =  0.212 mu=  -3.021 started at  2021-07-23 19:07:51.596436
T =  0.212 mu=  -2.9808 started at  2021-07-23 19:12:15.468696
T =  0.212 mu=  -2.9406 started at  2021-07-23 19:16:35.024140
T =  0.212 mu=  -2.9004 started at  2021-07-23 19:20:56.652445
T =  0.212 mu=  -2.8602 started at  2021-07-23 19:25:19.484787
T =  0.212 mu=  -2.82 started at  2021-07-23 19:29:46.526004
T =  0.212 mu=  -2.7798 started at  2021-07-23 19:34:09.104213
T =  0.212 mu=  -2.7396 started at  2021-07-23 19:38:28.613114
T =  0.212 mu=  -2.6994 started at  2021-07-23 19:42:51.077836
T =  0.212 mu=  -2.6592 started at  2021-07-23 19:47:11.670186
T =  0.212 mu=  -2.619 started at  2021-07-23 19:51:34.441763
T =  0.212 mu=  -2.5788 started at  2021-07-23 19:55:55.119282
T =  0.212 mu=  -2.5386 started at  2021-07-23 20:00:18.303673
T =  0.212 mu=  -2.4984 started at  2021-07-23 20:04:39.239842
T =  0.212 mu=  -2.4582 started at  2021-07-23 20:09:08.036684
T =  0.212 mu=  -2.418 started at  2021-07-23 20:13:26.3741

T =  0.212 mu=  2.2854 started at  2021-07-24 04:44:44.158135
T =  0.212 mu=  2.3256 started at  2021-07-24 04:49:02.194567
T =  0.212 mu=  2.3658 started at  2021-07-24 04:53:25.252321
T =  0.212 mu=  2.406 started at  2021-07-24 04:57:49.227303
T =  0.212 mu=  2.4462 started at  2021-07-24 05:02:10.574086
T =  0.212 mu=  2.4864 started at  2021-07-24 05:06:42.793275
T =  0.212 mu=  2.5266 started at  2021-07-24 05:11:06.585091
T =  0.212 mu=  2.5668 started at  2021-07-24 05:15:25.484861
T =  0.212 mu=  2.607 started at  2021-07-24 05:19:50.790713
T =  0.212 mu=  2.6472 started at  2021-07-24 05:24:17.519621
T =  0.212 mu=  2.6874 started at  2021-07-24 05:28:44.481285
T =  0.212 mu=  2.7276 started at  2021-07-24 05:33:11.534871
T =  0.212 mu=  2.7678 started at  2021-07-24 05:37:41.525665
T =  0.212 mu=  2.808 started at  2021-07-24 05:42:04.152308
T =  0.212 mu=  2.8482 started at  2021-07-24 05:46:25.910528
T =  0.212 mu=  2.8884 started at  2021-07-24 05:50:47.234053
T =  0.212 

T =  0.253 mu=  1.5216 started at  2021-07-24 14:15:20.292555
T =  0.253 mu=  1.5618 started at  2021-07-24 14:19:41.671056
T =  0.253 mu=  1.602 started at  2021-07-24 14:23:52.702558
T =  0.253 mu=  1.6422 started at  2021-07-24 14:28:02.318521
T =  0.253 mu=  1.6824 started at  2021-07-24 14:32:20.475973
T =  0.253 mu=  1.7226 started at  2021-07-24 14:36:37.835506
T =  0.253 mu=  1.7628 started at  2021-07-24 14:40:48.075033
T =  0.253 mu=  1.803 started at  2021-07-24 14:45:03.657919
T =  0.253 mu=  1.8432 started at  2021-07-24 14:49:10.843018
T =  0.253 mu=  1.8834 started at  2021-07-24 14:53:20.828002
T =  0.253 mu=  1.9236 started at  2021-07-24 14:57:32.530863
T =  0.253 mu=  1.9638 started at  2021-07-24 15:01:43.808507
T =  0.253 mu=  2.004 started at  2021-07-24 15:06:02.478208
T =  0.253 mu=  2.0442 started at  2021-07-24 15:10:16.314822
T =  0.253 mu=  2.0844 started at  2021-07-24 15:14:27.993116
T =  0.253 mu=  2.1246 started at  2021-07-24 15:18:48.833335
T =  0.253 

T =  0.294 mu=  0.7578 started at  2021-07-24 23:30:10.464689
T =  0.294 mu=  0.798 started at  2021-07-24 23:34:19.163127
T =  0.294 mu=  0.8382 started at  2021-07-24 23:38:31.149757
T =  0.294 mu=  0.8784 started at  2021-07-24 23:42:42.461891
T =  0.294 mu=  0.9186 started at  2021-07-24 23:46:54.372033
T =  0.294 mu=  0.9588 started at  2021-07-24 23:51:01.804470
T =  0.294 mu=  0.999 started at  2021-07-24 23:55:15.218985
T =  0.294 mu=  1.0392 started at  2021-07-24 23:59:31.387536
T =  0.294 mu=  1.0794 started at  2021-07-25 00:03:43.175899
T =  0.294 mu=  1.1196 started at  2021-07-25 00:07:53.637268
T =  0.294 mu=  1.1598 started at  2021-07-25 00:12:04.205385
T =  0.294 mu=  1.2 started at  2021-07-25 00:16:18.025194
T =  0.294 mu=  1.2402 started at  2021-07-25 00:20:29.530771
T =  0.294 mu=  1.2804 started at  2021-07-25 00:24:38.987893
T =  0.294 mu=  1.3206 started at  2021-07-25 00:28:54.393137
T =  0.294 mu=  1.3608 started at  2021-07-25 00:33:03.004999
T =  0.294 mu

T =  0.335 mu=  -0.006 started at  2021-07-25 08:44:21.511340
T =  0.335 mu=  0.0342 started at  2021-07-25 08:48:35.371142
T =  0.335 mu=  0.0744 started at  2021-07-25 08:52:52.252990
T =  0.335 mu=  0.1146 started at  2021-07-25 08:57:03.424034
T =  0.335 mu=  0.1548 started at  2021-07-25 09:01:10.423320
T =  0.335 mu=  0.195 started at  2021-07-25 09:05:19.578985
T =  0.335 mu=  0.2352 started at  2021-07-25 09:09:30.754020
T =  0.335 mu=  0.2754 started at  2021-07-25 09:13:42.174674
T =  0.335 mu=  0.3156 started at  2021-07-25 09:17:54.782464
T =  0.335 mu=  0.3558 started at  2021-07-25 09:22:08.957184
T =  0.335 mu=  0.396 started at  2021-07-25 09:26:19.992122
T =  0.335 mu=  0.4362 started at  2021-07-25 09:30:30.716130
T =  0.335 mu=  0.4764 started at  2021-07-25 09:34:43.606978
T =  0.335 mu=  0.5166 started at  2021-07-25 09:38:55.626069
T =  0.335 mu=  0.5568 started at  2021-07-25 09:43:07.698063
T =  0.335 mu=  0.597 started at  2021-07-25 09:47:25.780194
T =  0.335 

T =  0.376 mu=  -0.7698 started at  2021-07-25 18:02:40.727142
T =  0.376 mu=  -0.7296 started at  2021-07-25 18:06:53.745709
T =  0.376 mu=  -0.6894 started at  2021-07-25 18:11:04.214032
T =  0.376 mu=  -0.6492 started at  2021-07-25 18:15:20.068656
T =  0.376 mu=  -0.609 started at  2021-07-25 18:19:34.082757
T =  0.376 mu=  -0.5688 started at  2021-07-25 18:23:47.646120
T =  0.376 mu=  -0.5286 started at  2021-07-25 18:28:01.855228
T =  0.376 mu=  -0.4884 started at  2021-07-25 18:32:17.841410
T =  0.376 mu=  -0.4482 started at  2021-07-25 18:36:41.081396
T =  0.376 mu=  -0.408 started at  2021-07-25 18:40:56.892173
T =  0.376 mu=  -0.3678 started at  2021-07-25 18:45:15.313239
T =  0.376 mu=  -0.3276 started at  2021-07-25 18:49:31.064325
T =  0.376 mu=  -0.2874 started at  2021-07-25 18:53:42.979551
T =  0.376 mu=  -0.2472 started at  2021-07-25 18:57:53.668955
T =  0.376 mu=  -0.207 started at  2021-07-25 19:02:11.620786
T =  0.376 mu=  -0.1668 started at  2021-07-25 19:06:30.17

T =  0.417 mu=  -1.5336 started at  2021-07-26 03:25:56.386508
T =  0.417 mu=  -1.4934 started at  2021-07-26 03:30:14.958908
T =  0.417 mu=  -1.4532 started at  2021-07-26 03:34:31.265151
T =  0.417 mu=  -1.413 started at  2021-07-26 03:38:44.050498
T =  0.417 mu=  -1.3728 started at  2021-07-26 03:42:58.371105
T =  0.417 mu=  -1.3326 started at  2021-07-26 03:47:21.181134
T =  0.417 mu=  -1.2924 started at  2021-07-26 03:51:38.831744
T =  0.417 mu=  -1.2522 started at  2021-07-26 03:55:50.805312
T =  0.417 mu=  -1.212 started at  2021-07-26 04:00:05.686912
T =  0.417 mu=  -1.1718 started at  2021-07-26 04:04:16.192371
T =  0.417 mu=  -1.1316 started at  2021-07-26 04:08:32.383086
T =  0.417 mu=  -1.0914 started at  2021-07-26 04:12:50.429515
T =  0.417 mu=  -1.0512 started at  2021-07-26 04:17:09.417226
T =  0.417 mu=  -1.011 started at  2021-07-26 04:21:23.525152
T =  0.417 mu=  -0.9708 started at  2021-07-26 04:25:46.175099
T =  0.417 mu=  -0.9306 started at  2021-07-26 04:30:05.22